<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#damage_zones(-)-to-calculate-parameters-of-damage-zones" data-toc-modified-id="damage_zones(-)-to-calculate-parameters-of-damage-zones-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>damage_zones( ) to calculate parameters of damage zones</a></span></li><li><span><a href="#impact_risk(-)-uncertainty-analysis" data-toc-modified-id="impact_risk(-)-uncertainty-analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>impact_risk( ) uncertainty analysis</a></span></li></ul></div>

In [4]:
from armageddon.damage import damage_zones, impact_risk

To install the module and any pre-requisites, from the base directory run:
```
pip install -r requirements.txt
pip install -e .
```

To download the postcode data required:

```
python download_data.py
```

### damage_zones( ) to calculate parameters of damage zones

Calculate the latitude and longitude of the surface zero location and the
list of airblast damage radii (m) for a given impact scenario.


__Parameters__

outcome: Dict

    the outcome dictionary from an impact scenario
        
lat: float

    latitude of the meteoroid entry point (degrees)

lon: float

    longitude of the meteoroid entry point (degrees)
    
bearing: float

    Bearing (azimuth) relative to north of meteoroid trajectory (degrees)
    
pressures: float, arraylike

    List of threshold pressures to define airblast damage levels
    
plot: bool

    Boolean value to decide plotting

__Return__

blat: float

    latitude of the surface zero point (degrees)
        
blon: float

    longitude of the surface zero point (degrees)
        
damrad: arraylike, float

    List of distances specifying the blast radii for the input damage levels
        
plot: plot object

    The plot specifying the areas effected by each damage level


__Examples__

    >>> a = [[2, 0, -1], [0, 5, 6], [0, -1, 1]]
    >>> d = det(a)
    >>> d
    22.0
    
    >>> import armageddon
    >>> outcome = {'burst_altitude': 8e3, 'burst_energy': 7e3,
                   'burst_distance': 90e3, 'burst_peak_dedz': 1e3,
                   'outcome': 'Airburst'}
    >>> armageddon.damage_zones(outcome, 52.79, -2.95, 135,
                                pressures=[1e3, 3.5e3, 27e3, 43e3])
    (51.98371949678459, -2.746093100475223, [51002.04019315775,
     25706.014907978413, 7196.208816950395, 3803.4573273729134])


In [5]:
outcome = {'burst_altitude': 8e3, 'burst_energy': 7e3,
                   'burst_distance': 90e3, 'burst_peak_dedz': 1e3,
                   'outcome': 'Airburst'}
damage_zones(outcome, 52.79, -2.95, 135, pressures=[1e3, 3.5e3, 27e3, 43e3])

(51.98371949678459,
 -2.746093100475223,
 [51002.04019315775,
  25706.014907978413,
  7196.208816950395,
  3803.4573273729134])

### impact_risk( ) uncertainty analysis

Perform an uncertainty analysis to calculate the risk for each affected UK postcode or postcode sector


__Parameters__

planet: armageddon.Planet instance

    The Planet instance from which to solve the atmospheric entry

means: dict

    A dictionary of mean input values for the uncertainty analysis. This should include values for ``radius``, ``angle``, ``strength``, ``density``, ``velocity``, ``lat``, ``lon`` and ``bearing``

stdevs: dict

    A dictionary of standard deviations for each input value. This should include values for ``radius``, ``angle``, ``strength``, ``density``, ``velocity``, ``lat``, ``lon`` and ``bearing``

pressure: float

    A single pressure at which to calculate the damage zone for each impact

nsamples: int

    The number of iterations to perform in the uncertainty analysis

sector: logical, optional

    If True (default) calculate the risk for postcode sectors, otherwise calculate the risk for postcodes

__Return__

risk: DataFrame

    A pandas DataFrame with columns for postcode (or postcode sector) and the associated risk. These should be called ``postcode`` or ``sector``, and ``risk``.
